<a href="https://colab.research.google.com/github/jpos92/projeto_imersao_ia/blob/master/Projeto_imers%C3%A3o_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai
!pip install pypdf2

In [ ]:
# Importando recursos necessários para a aplicação.
import google.generativeai as genai
from google.colab import userdata
from google.colab import files
import PyPDF2

# Definição de variáveis gerais
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

#Definição e configurações do modelo
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# UPLOAD DE ARQUIVOS

In [ ]:
uploaded = files.upload()

Saving EDITAL_LICITACAO_XX_2023.pdf to EDITAL_LICITACAO_XX_2023.pdf
Saving Esperanca_SA.pdf to Esperanca_SA.pdf
Saving Medical_Corp.pdf to Medical_Corp.pdf
Saving Saude_Ltd.pdf to Saude_Ltd.pdf
Saving Tabela_Estoque_Anterior.pdf to Tabela_Estoque_Anterior.pdf
Saving Tabela_Medicamentos_Orgao_Regulador.pdf to Tabela_Medicamentos_Orgao_Regulador.pdf


# FUNCÃO PARA LER PDF

In [ ]:
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  with open(pathname, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)

    for page_num in range(num_pages):
      page = pdf_reader.pages[page_num]
      page_content = page.extract_text()
      parts.append(f"--- PAGE {page_num} ---")
      parts.append(page_content)

  return parts

# MÓDULO PARA AVALIAR EMPRESAS

In [ ]:
prompt_parts = [
  *extract_pdf_pages("/content/Esperanca_SA.pdf"),
  *extract_pdf_pages("/content/Medical_Corp.pdf"),
  *extract_pdf_pages("/content/Saude_Ltd.pdf"),
  "\"Você é um sistema construído para trabalhar no contexto de licitações públicas\", \"Uma empresa só pode ser desclassificada se estiver envolvida em processos por corrupção, apenas nesse caso\", \"Faça uma análise das empresas e responda com uma tabela contendo o nome da empresa, status 'Positiva', 'Negativa' ou 'Indiferente' e se está ou não classificada\", \"Faça um pequeno resumo com uma visão geral de cada empresa\""
]

response = model.generate_content(prompt_parts)
print(response.text)

## Análise de Empresas para Licitação Pública

| Nome da Empresa | Status | Classificada? |
|---|---|---|
| Esperança S.A. | Positiva | Sim |
| Medical Corp. | Indiferente | Sim |
| Saúde Ltd. | Negativa | Não |

## Resumo das Empresas

**Esperança S.A.:** Apresenta um perfil exemplar, sem envolvimento em processos judiciais e com documentação completa. É uma candidata ideal para a licitação.

**Medical Corp.:** Apesar de não ter processos por corrupção, a empresa enfrenta problemas trabalhistas e relacionados a discursos impróprios. Embora possa participar da licitação, é importante considerar esses fatores na avaliação.

**Saúde Ltd.:** O envolvimento em processos de corrupção desclassifica automaticamente a empresa da licitação. Os demais processos e pendências documentais reforçam ainda mais a sua inadequação para o processo. 



# MÓDULO PARA AVALIAR POSSÍVEL TENTATIVA DE FRAUDE COM LICITAÇÕES INJUSTIFICADAS

In [ ]:
prompt_parts = [
  *extract_pdf_pages("/content/Tabela_Estoque_Anterior.pdf"),
  "\"A abertura de um processo licitatório só pode ser iniciado se mais da metade dos medicamentos estiverem com porcentagem menor ou igual a 50%\""
]

response = model.generate_content(prompt_parts)
print(response.text)

## Análise do Estoque de Medicamentos e Necessidade de Licitação

Com base na tabela de estoque de medicamentos e na regra de abertura de processo licitatório, vamos analisar a situação:

**Medicamentos com porcentagem menor ou igual a 50%:**

* **Ibuprofeno 200 mg:** 50,00%
* **Amoxilina 500 mg:** 50,00%
* **Azitromicina 500 mg:** 35,00%
* **Cefalexina 500 mg:** 37,50%
* **Ciprofloxacino 500 mg:** 36,67%
* **Levofloxacino 500 mg:** 15,00%
* **Clindamicina 300 mg:** 5,00%

**Total de medicamentos:** 10

**Medicamentos com porcentagem menor ou igual a 50%:** 7 

**Conclusão:**

Como 7 dos 10 medicamentos (70%) possuem uma porcentagem menor ou igual a 50%, **a condição para abertura de um processo licitatório é atendida.** É crucial iniciar o processo de licitação o mais rápido possível para evitar a falta de medicamentos essenciais e garantir o abastecimento adequado. 



# MÓDULO PARA AVALIAR POSSÍVEL TENTATIVA DE FRAUDE ATRAVÉS DE SUPERFATURAMENTO

In [ ]:
prompt_parts = [
  *extract_pdf_pages("/content/Tabela_Medicamentos_Orgao_Regulador.pdf"),
  *extract_pdf_pages("/content/EDITAL_LICITACAO_XX_2023.pdf"),
  "\"Compare os preços do edital com os preços da tabela oficial do órgão regulador e indique possiveis fraudes\""
]

response = model.generate_content(prompt_parts)
print(response.text)

## Análise de Possíveis Fraudes no Edital de Licitação XX/2023

Comparando os preços unitários do edital com a tabela oficial do órgão regulador, identificamos discrepâncias significativas em alguns itens, sugerindo possíveis fraudes:

**Itens com Discrepâncias:**

* **AAS 100mg:** O edital apresenta um valor unitário de R$ 0,48, enquanto a tabela oficial indica R$ 0,12. **Isso representa uma diferença de 300%,** o que é um forte indício de sobrepreço.
* **Cefalexina 500mg:** O edital indica um valor unitário de R$ 0,70, enquanto a tabela oficial apresenta R$ 0,30. **A diferença de 133%** também levanta suspeitas de sobrepreço.

**Outros pontos a serem considerados:**

* **Margem de Lucro Elevada:** A margem de lucro de 20% estabelecida no edital pode ser considerada alta para medicamentos, especialmente considerando que se trata de uma compra em grande quantidade para uso hospitalar. 
* **Ausência de Justificativa:** O edital não apresenta nenhuma justificativa para as diferenças de p